### Spotify Login
After running this click Sign-In and login to Spotify. If the Time expires then the token should refresh using the popup. 

In [ ]:
%load_ext autoreload
%autoreload 2
from ipyauth import ParamsSpotify, Auth
from spotify import Spotify

auth = Auth(ParamsSpotify(redirect_uri='http://localhost:8888/callback', client_id="9e4657eefbac41afa98c61f590d8fd51"))
auth

### Common Stuff and Imports


In [ ]:
import requests
from IPython.display import Image

from pandas import DataFrame,read_pickle,merge
from pandas import DataFrame as df
from datetime import datetime,timedelta,date
import json

import logging
import os


logging.basicConfig()
defaultLogger = logging.getLogger()
defaultLogger.setLevel(logging.ERROR)
logger = logging.getLogger('SpotifyLab')
logger.setLevel(logging.DEBUG)

spot = Spotify(auth, logger = logger)

Image(url=spot.imageUrl, width=100)


### Read The User Library
This process may take a little while. The library tracks are cached locally, so this step can be skipped.

In [ ]:
from pandas.io.json import json_normalize

data = spot.fetchLibrary()
tracksDf = json_normalize(data, sep="_").set_index("track_uri")
tracksDf.to_pickle("mytracks.pkl")
tracksDf.head(2)

### Verify the library cache exists

In [ ]:
from pandas.io.json import json_normalize
import pandas as pd
import math
import numpy as np

# Read Cached File

tracksDf = read_pickle("mytracks.pkl").reset_index()

# Add Original URI and ID for linking
tracksDf["original_id"] = tracksDf.apply(lambda t: t["track_id"] if (pd.isnull(t["track_linked_from_id"])) else t["track_linked_from_id"], axis=1 )
tracksDf["original_uri"] = tracksDf.apply(lambda t: t["track_uri"] if (pd.isnull(t["track_linked_from_uri"])) else t["track_linked_from_uri"], axis=1 )
tracksDf = tracksDf.set_index("original_uri")

# Read Albums, with cache
albumsPickle = read_pickle("albums.pkl") if (os.path.isfile("albums.pkl")) else None 
album_ids  = list(set(tracksDf["track_album_id"].values))
albums = spot.fetchAllIds("/v1/albums","albums",album_ids,pageSize=20,existingDf=albumsPickle)
albumsDf = json_normalize(albums, sep="_").set_index("id")
albumsDf.to_pickle("albums.pkl")

# Add a Released DateTime Column, calculated from the release_date
albumsDf["released"] = albumsDf.apply(lambda al: datetime.strptime(al["release_date"], "%Y" if (al.release_date_precision=="year") else "%Y-%m" if (al.release_date_precision=="month") else "%Y-%m-%d"), axis=1) 

# Join the Albums Columns using track_album_id index to album
libraryWithAlbums = merge(tracksDf,albumsDf, left_on="track_album_id", right_index=True, suffixes=("_track","_album"))

# Add an Artist Name column, first in the album list
libraryWithAlbums["artist"] = libraryWithAlbums.apply(lambda a: a["artists"][0]["name"], axis=1 )

# Read the Features
featuresPickle = read_pickle("features.pkl").set_index("id") if (os.path.isfile("features.pkl")) else None 
features = spot.fetchAllIds("/v1/audio-features","audio_features",tracksDf["original_id"].values,pageSize=50,existingDf=featuresPickle)
featuresDf = json_normalize(features, sep="_")
featuresDf.to_pickle("features.pkl")

# REMOVED - ARTIST FETCHING - Add BACK LATER
#artist_and_track = json_normalize( data=data, record_path=['track','artists'],  meta=[["track","name"],["track","uri"]],  record_prefix='artist_',   sep="_" )
#artist_and_track = artist_and_track[['track_name','artist_id','artist_name', 'track_uri']]
#artistsPickle = read_pickle("artists.pkl") if (os.path.isfile("artists.pkl")) else None 
#artistIds = list(set(artist_and_track["artist_id"].values))
#artists = spot.fetchAllIds("/v1/artists","artists",artistIds,existingDf=artistsPickle)
#artistsDf = json_normalize(artists).set_index("id")
#artistsDf.to_pickle("artists.pkl")
#artistsDf[["name","genres"]].head(2)

# Merge Features
lib = merge(libraryWithAlbums,featuresDf.set_index("uri"), left_index=True, right_index=True, how="outer")
lib.columns.values
#lib[lib.track_name=="Spooky - Out of Order Mix"][["artist","track_name","tempo","danceability","loudness","energy","released","valence"]].head(2)

---------------------------------

## All Read - let's create an auto playlist



### Running Playlist 
* Tempo between 160 and 200 (for cadence)
* Energy above 0.6
* Danceability above 0.7

In [ ]:
newPlaylist = lib[(lib.energy>0.90) & (lib.tempo>100) & (lib.instrumentalness>0.5) & (lib.duration_ms<1000*60*6) & (lib.danceability>0.4) & (lib.loudness>-11)]
newPlaylist = newPlaylist[~newPlaylist.artist.isin(["Rodrigo y Gabriela","2CELLOS","Faithless","Doves"])]
spot.updatePlaylist("Auto Run Fast",newPlaylist)


In [ ]:
spot.updatePlaylist("Are You Sure",lib[lib.track_popularity<0.10])


In [ ]:
spot.updatePlaylist("auto 1960s",lib[lib.released<datetime(1970,1,1)])
spot.updatePlaylist("auto 1970s",lib[(lib.released>datetime(1970,1,1)) & (lib.released<datetime(1980,1,1))])
spot.updatePlaylist("auto 1980s",lib[(lib.released>datetime(1980,1,1)) & (lib.released<datetime(1990,1,1))])
spot.updatePlaylist("auto 1990s",lib[(lib.released>datetime(1990,1,1)) & (lib.released<datetime(2000,1,1))])
spot.updatePlaylist("auto 2000s",lib[(lib.released>datetime(2000,1,1)) & (lib.released<datetime(2010,1,1))])
spot.updatePlaylist("auto 2010s",lib[(lib.released>datetime(2010,1,1)) & (lib.released<datetime(2020,1,1))])
spot.updatePlaylist("2019 So Far",lib[lib.released>datetime(2019,1,1)] )


In [ ]:
spot.updatePlaylist("Auto Recent",lib[lib.added_at>(datetime.now()-timedelta(days=60)).isoformat()])


In [ ]:
spot.updatePlaylist("Dinner Party",lib[(lib.acousticness>0.9) & (lib.energy<0.6)  ])
spot.updatePlaylist("Night Nights",lib[(lib.acousticness>0.9) & (lib.energy<0.3) & (lib.danceability<0.3)])
spot.updatePlaylist("To Work To",lib[(lib.instrumentalness>0.9) & (lib.energy>0.6) ])
spot.updatePlaylist("Quiet Work",lib[(lib.instrumentalness>0.9) & (lib.danceability<0.6) & (lib.energy>0.3) ])

In [ ]:
lib[lib.track_is_playable==False][["track_name","artist","released","track_is_local"]].head(67)

In [ ]:
# Duplicates in the library
ff = lib.groupby(["track_name","artist"]).count()
(ff[ff.track_uri>1]).index.values